In [1]:
import numpy as np
import tensorflow as tf


print(tf.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

2.3.2


In [2]:
dataset = np.load("dataset_overlap.npy")
classes = np.load("/home/andrea/Scrivania/MIVIA_ROAD_DB1/bin_classes.npy")

In [3]:
def unison_shuffled_copies(a, b):
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [4]:
def split(ds, labels: np.ndarray, perc):
    split_point = int(np.ceil(len(ds)*perc))
    val_dim = len(ds)-split_point
    ones = 0
    zeros = 0
    train_inserted = 0
    val_inserted = 0
    train_ds = np.zeros((split_point,32,16516))
    train_labels = np.zeros((split_point,), dtype=np.int64)
    val_ds = np.zeros((val_dim, 32, 16516))
    val_labels = np.zeros((val_dim,),  dtype=np.int64)
    for data, label in zip(ds,labels):
        if label == 0:
            if zeros < np.ceil(split_point/2):
                train_ds[train_inserted] = data
                train_labels[train_inserted] = label
                zeros +=1
                train_inserted += 1
            else:
                val_ds[val_inserted] = data
                val_labels[val_inserted] = label
                val_inserted += 1
        if label == 1:
            if ones < np.ceil(split_point/2):
                train_ds[train_inserted] = data
                train_labels[train_inserted] = label
                ones +=1
                train_inserted += 1
            else:
                val_ds[val_inserted] = data
                val_labels[val_inserted] = label
                val_inserted += 1

    return train_ds, train_labels, val_ds, val_labels

In [5]:
def normalize(array, std_dev, mean, scale):
    array = array/scale
    return array-mean/std_dev

In [27]:
def split_dataset_and_normalize(perc=0.8):
    shuffled_dataset, shuffled_labels = unison_shuffled_copies(dataset,classes)
    split_point = int(len(dataset)*perc)
    train_ds, train_labels, val_ds, val_labels = split(shuffled_dataset, shuffled_labels, perc)
    std_dev = np.std(train_ds)
    mean = np.mean(train_ds)
    maximum = np.max(train_ds)
    min = np.min(train_ds)
    scale = max(maximum, abs(min))
    train_ds = normalize(train_ds, std_dev, mean, scale)
    val_ds = normalize(val_ds, std_dev, mean, scale)
    return train_ds, train_labels, val_ds, val_labels

train_ds, train_labels, val_ds, val_labels = split_dataset_and_normalize()

In [7]:
print((train_labels == 0).sum())
print((train_labels == 1).sum())
print((val_labels == 0).sum())
print((val_labels == 1).sum())

297
297
45
103


BIDIRECTIONAL LSTM

In [2]:
from tensorflow.keras.layers import LSTM, Dense, Dropout, InputLayer, GRU, LSTMCell, RNN, Bidirectional
from tensorflow import keras
from tensorflow.keras import  regularizers
input_shape=(32,16516)
model = keras.Sequential()
model.add(InputLayer(input_shape=input_shape))
model.add(Bidirectional(LSTM(192, input_shape=input_shape,
                             kernel_regularizer=regularizers.l2(0.00001),
                                 recurrent_regularizer=regularizers.l2(0.00001),
                                 bias_regularizer=regularizers.l2(0.00001),
                             recurrent_dropout=0.3)))
model.add(Dropout(0.4))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 384)               25665024  
_________________________________________________________________
dropout (Dropout)            (None, 384)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               49280     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2

BIDIRECTIONAL LSTM #2

In [72]:
from tensorflow.keras.layers import LSTM, Dense, Dropout, InputLayer, GRU, LSTMCell, RNN, Bidirectional
from tensorflow import keras
from tensorflow.keras import  regularizers
input_shape=(32,16516)
model = keras.Sequential()
model.add(InputLayer(input_shape=input_shape))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(192, input_shape=input_shape,
                             kernel_regularizer=regularizers.l2(0.00001),
                                 recurrent_regularizer=regularizers.l2(0.00001),
                                 bias_regularizer=regularizers.l2(0.00001),
                             recurrent_dropout=0.4)))
model.add(Dropout(0.4))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 32, 4096)          67653632  
_________________________________________________________________
dropout_37 (Dropout)         (None, 32, 4096)          0         
_________________________________________________________________
dense_38 (Dense)             (None, 32, 2048)          8390656   
_________________________________________________________________
dropout_38 (Dropout)         (None, 32, 2048)          0         
_________________________________________________________________
dense_39 (Dense)             (None, 32, 1024)          2098176   
_________________________________________________________________
dropout_39 (Dropout)         (None, 32, 1024)          0         
_________________________________________________________________
dense_40 (Dense)             (None, 32, 512)          

1-DIR LSTM #1

In [89]:
from tensorflow.keras.layers import LSTM, Dense, Dropout, InputLayer, GRU, LSTMCell, RNN, Bidirectional
from tensorflow import keras
from tensorflow.keras import  regularizers
input_shape=(32,16516)
model = keras.Sequential()
model.add(InputLayer(input_shape=input_shape))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(LSTM(128, input_shape=input_shape,
                             kernel_regularizer=regularizers.l2(0.00001),
                                 recurrent_regularizer=regularizers.l2(0.00001),
                                 bias_regularizer=regularizers.l2(0.00001),
                             recurrent_dropout=0.4))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 128)               8522240   
_________________________________________________________________
dropout_49 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_49 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_50 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_50 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_51 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_51 (Dense)             (None, 1)               

1-DIR LSTM #2

In [99]:
from tensorflow.keras.layers import LSTM, Dense, Dropout, InputLayer, GRU, LSTMCell, RNN, Bidirectional
from tensorflow import keras
from tensorflow.keras import  regularizers
input_shape=(32,16516)
model = keras.Sequential()
model.add(InputLayer(input_shape=input_shape))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(LSTM(96, input_shape=input_shape,
                             kernel_regularizer=regularizers.l2(0.000001),
                                 recurrent_regularizer=regularizers.l2(0.000001),
                                 bias_regularizer=regularizers.l2(0.000001),
                             recurrent_dropout=0.4))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 96)                6379392   
_________________________________________________________________
dropout_52 (Dropout)         (None, 96)                0         
_________________________________________________________________
dense_52 (Dense)             (None, 64)                6208      
_________________________________________________________________
dropout_53 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_53 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_54 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_54 (Dense)             (None, 1)               

1-DIR LSTM #3

In [126]:
from tensorflow.keras.layers import LSTM, Dense, Dropout, InputLayer, GRU, LSTMCell, RNN, Bidirectional
from tensorflow import keras
from tensorflow.keras import  regularizers
input_shape=(32,16516)
model = keras.Sequential()
model.add(InputLayer(input_shape=input_shape))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(LSTM(64, input_shape=input_shape,
                             kernel_regularizer=regularizers.l2(0.00005),
                                 recurrent_regularizer=regularizers.l2(0.00005),
                                 bias_regularizer=regularizers.l2(0.00005),
                             recurrent_dropout=0.4))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_12 (LSTM)               (None, 64)                4244736   
_________________________________________________________________
dropout_55 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_55 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_56 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_56 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_57 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_57 (Dense)             (None, 1)               

In [127]:
filepath = "audio_model_LSTM_overlap_3.h5"

In [128]:
cb1 = tf.keras.callbacks.ModelCheckpoint(
    filepath, monitor='val_loss', verbose=0, save_best_only=True,
    save_weights_only=False, mode='auto', save_freq='epoch',
    options=None
)
cb2 = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

cbs = [cb1,cb2]

In [3]:
#callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

In [130]:
EPOCHS = 50
history = model.fit(
    x= train_ds,
    y = train_labels,
    validation_data=(val_ds, val_labels),
    batch_size=16,
    epochs=EPOCHS,
    callbacks=cbs
)

Epoch 1/50
38/38 [==============================] - 4s 92ms/step - loss: 0.7237 - accuracy: 0.4949 - val_loss: 0.7227 - val_accuracy: 0.4392

In [125]:
del model

In [131]:
np.save('history_no_bid3.npy',history.history)

In [122]:
model = keras.models.load_model("models/audio_model_LSTM_overlap.h5")

In [12]:
model.evaluate(x=val_ds, y=val_labels)

10/10 [==============================] - 0s 35ms/step - loss: 0.0311 - accuracy: 0.9730


[0.031140539795160294, 0.9729729890823364]

In [56]:
del model2

In [ ]:
del train_ds, train_labels, val_ds, val_labels

RESULTS MATRIX

In [132]:
labels_pred_train = model.predict_classes(train_ds).flatten()

In [133]:
tp, fp, tn, fn = 0,0,0,0
for y_true, y_pred in zip(train_labels, labels_pred_train):
    if y_true == 1 and y_pred == 1:
        tp+=1
    if y_true == 0 and y_pred == 0:
        tn+=1
    if y_true == 1 and y_pred == 0:
        fn+=1
    if y_true == 0 and y_pred == 1:
        fp+=1
positives = (train_labels == 1).sum()
negatives = (train_labels == 0).sum()
print("tp: " +str(round((tp/positives)*100,2)) + "%" +
      "  tn: " +str(round((tn/negatives)*100,2)) + "%" +
      "  fp: " +str(round((fp/negatives)*100,2)) + "%" +
      "  fn: " +str(round((fn/positives)*100,2)) + "%")

tp: 94.95%  tn: 99.33%  fp: 0.67%  fn: 5.05%


In [134]:
labels_pred_val = model.predict_classes(val_ds)

In [135]:
tp, fp, tn, fn = 0,0,0,0
for y_true, y_pred in zip(val_labels, labels_pred_val):
    if y_true == 1 and y_pred == 1:
        tp+=1
    if y_true == 0 and y_pred == 0:
        tn+=1
    if y_true == 1 and y_pred == 0:
        fn+=1
    if y_true == 0 and y_pred == 1:
        fp+=1

positives = (val_labels == 1).sum()
negatives = (val_labels == 0).sum()
print("tp: " +str(round((tp/positives)*100,2)) + "%" +
      "  tn: " +str(round((tn/negatives)*100,2)) + "%" +
      "  fp: " +str(round((fp/negatives)*100,2)) + "%" +
      "  fn: " +str(round((fn/positives)*100,2)) + "%")

tp: 71.84%  tn: 93.33%  fp: 6.67%  fn: 28.16%


In [64]:
hs = np.load("history.npy", allow_pickle=True)